In [1]:
import os.path
import torch


try:
    from google.colab import drive
    drive.mount('/content/gdrive')
    _home = 'gdrive/MyDrive/'
except ImportError:
    _home = '~'
finally:
    data_root = os.path.join(_home, '.pytorch')

print(data_root)


#getting yolo model from ultralytics and installing all dependecies
!git clone https://github.com/ultralytics/yolov5  # clone
%cd yolov5
%pip install -qr requirements.txt  # install
import utils
display = utils.notebook_init()

YOLOv5 🚀 v7.0-120-g3e55763 Python-3.9.16 torch-1.13.1+cu116 CUDA:0 (Tesla T4, 15102MiB)


Setup complete ✅ (2 CPUs, 12.7 GB RAM, 25.4/78.2 GB disk)


In [ ]:
#prearing TensorBoard logging, can be switched to CML and Comet
logger = 'TensorBoard' #@param ['ClearML', 'Comet', 'TensorBoard']

if logger == 'ClearML':
  %pip install -q clearml
  import clearml; clearml.browser_login()
elif logger == 'Comet':
  %pip install -q comet_ml
  import comet_ml; comet_ml.init()
elif logger == 'TensorBoard':
  %load_ext tensorboard
  %tensorboard --logdir runs/train

In [ ]:
# Download COCO val
torch.hub.download_url_to_file('https://ultralytics.com/assets/coco2017val.zip', 'tmp.zip')  # download (780M - 5000 images)
!unzip -q tmp.zip -d ../datasets && rm tmp.zip  # unzip

In [ ]:
#Evaluate before training
!python val.py --weights yolov5s.pt --data coco.yaml --img 640 

In [ ]:
'''#calling yolo training script from /yolov5 directory, downloads coco128 set, with small yv5 network, caches weights and paramters to
disk and ram for faster performance'''
!python train.py --img 640 --batch 16 --epochs 10 --data coco128.yaml --weights yolov5s.pt 

In [ ]:
#evaluate after training, using best weights from training, change current "expX" folder to your current training run folder
!python val.py --weights runs/train/exp1/weights/best.pt --data coco.yaml --img 640 